https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset

In [6]:
import pandas as pd
import numpy as np  
import ast


#aisles = pd.read_csv(r'kaggle/archive/aisles.csv')
#departments = pd.read_csv(r'kaggle/archive/departments.csv')
prior = pd.read_csv(r'kaggle/archive/order_products__prior.csv')
train = pd.read_csv(r'kaggle/archive/order_products__train.csv')
orders = pd.read_csv(r'kaggle/archive/orders.csv')
products = pd.read_csv(r'kaggle/archive/products.csv')
mapping = pd.read_csv(r'simple_product_mapping.csv')
off = pd.read_csv(r'OpenFoodData.csv')
#print(aisles.head())
#print(departments.head())
print(prior.head())
print(train.head())
print(orders.head())
print(products.head())
#products = pd.merge(products, aisles, on='aisle_id')
#products = pd.merge(products, departments, on='department_id')

print(prior.shape)
print(train.shape)
print(orders.shape)


   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4 

In [7]:
#prior = pd.merge(prior,products, on='product_id', how='left')
#train = pd.merge(train,products, on='product_id', how='left')
prior = pd.merge(prior, mapping, left_on='product_id',right_on='kaggle_product_id', how='left')
train = pd.merge(train, mapping, left_on='product_id',right_on='kaggle_product_id', how='left')
prior = prior[['order_id', 'off_product_id']]
train = train[['order_id', 'off_product_id']]
prior = prior[prior['off_product_id']!='NO_MATCH']
train = train[train['off_product_id']!='NO_MATCH']

category_dict = {
    str(row['Id']): 
    ast.literal_eval(str(row['Categories']))
    for _, row in off.iterrows()
}
prior['aisle_id'] = prior['off_product_id'].map(category_dict).apply(list)
train['aisle_id'] = train['off_product_id'].map(category_dict).apply(list)


prior = prior.groupby('order_id').agg({
'off_product_id': list,
'aisle_id': list
}).reset_index()
train = train.groupby('order_id').agg({
'off_product_id': list,
'aisle_id': list
}).reset_index()
orders = orders[orders['eval_set'] != 'test'] # this is data in kaggle to put into model to predict
orders = orders[['order_id', 'user_id', 'days_since_prior_order','order_number']]
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)
orders = pd.merge(orders, prior, on='order_id', how='left')
orders = pd.merge(orders, train, on='order_id', how='left')

print(orders.head())

   order_id  user_id  days_since_prior_order  order_number  \
0   2539329        1                     0.0             1   
1   2398795        1                    15.0             2   
2    473747        1                    21.0             3   
3   2254736        1                    29.0             4   
4    431534        1                    28.0             5   

                                    off_product_id_x  \
0  [5902802802576, 25026290, 5908230536007, 35770...   
1  [5902802802576, 8711327672499, 5908230536007, ...   
2  [5902802802576, 5908230536007, 8711327672499, ...   
3  [5902802802576, 5908230536007, 8711327672499, ...   
4  [5902802802576, 5908230536007, 8711327672499, ...   

                                          aisle_id_x off_product_id_y  \
0  [[Dodatki do żywności, Środki przeciwzbrylając...              NaN   
1  [[Dodatki do żywności, Środki przeciwzbrylając...              NaN   
2  [[Dodatki do żywności, Środki przeciwzbrylając...              NaN  

In [11]:
orders['off_product_id'] = orders['off_product_id_x'].fillna(orders['off_product_id_y'])
orders['aisle_id_x'] = orders['aisle_id_x'].fillna(orders['aisle_id_y'])
#orders['department_id_x'] = orders['department_id_x'].fillna(orders['department_id_y'])
orders.rename(columns={'off_product_id_x': 'off_product_id'}, inplace=True)
orders.rename(columns={'aisle_id_x': 'aisle_id'}, inplace=True)
orders.drop('off_product_id_y', axis=1, inplace=True)
orders.drop(['aisle_id_y'], axis=1, inplace=True)
# thinking about whether to keep department_id or not
#orders.drop('department_id_x', axis=1, inplace=True)

print(orders.iloc[0,:])
print(orders.shape)
print(orders.columns)

order_id                                                            2539329
user_id                                                                   1
days_since_prior_order                                                  0.0
order_number                                                              1
off_product_id            [5902802802576, 25026290, 5908230536007, 35770...
aisle_id                  [[Dodatki do żywności, Środki przeciwzbrylając...
off_product_id            [5902802802576, 25026290, 5908230536007, 35770...
Name: 0, dtype: object
(3346083, 7)
Index(['order_id', 'user_id', 'days_since_prior_order', 'order_number',
       'off_product_id', 'aisle_id', 'off_product_id'],
      dtype='object')


# Sort values and then drop column

In [12]:
orders = orders.sort_values(by = ['user_id', 'order_number'],ascending=[True,True])
orders.drop('order_number', axis=1, inplace=True)

# Save to file

In [13]:
orders.to_csv('../ml/data/kaggle_prepared.csv', index=False)